In [1]:
# docs and experiment results can be found at https://docs.cleanrl.dev/rl-algorithms/dqn/#dqnpy
import os
import random
import time
from dataclasses import dataclass

import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tyro
from torch.utils.tensorboard import SummaryWriter

from cleanrl_utils.buffers import ReplayBuffer


@dataclass
class Args:
    # exp_name: str = os.path.basename(__file__)[: -len(".py")]
    exp_name: str = "dqn"
    """the name of this experiment"""
    seed: int = 1
    """seed of the experiment"""
    torch_deterministic: bool = True
    """if toggled, `torch.backends.cudnn.deterministic=False`"""
    cuda: bool = True
    """if toggled, cuda will be enabled by default"""
    track: bool = False
    """if toggled, this experiment will be tracked with Weights and Biases"""
    wandb_project_name: str = "cleanRL"
    """the wandb's project name"""
    wandb_entity: str = None
    """the entity (team) of wandb's project"""
    capture_video: bool = False
    """whether to capture videos of the agent performances (check out `videos` folder)"""
    save_model: bool = False
    """whether to save model into the `runs/{run_name}` folder"""
    upload_model: bool = False
    """whether to upload the saved model to huggingface"""
    hf_entity: str = ""
    """the user or org name of the model repository from the Hugging Face Hub"""

    # Algorithm specific arguments
    env_id: str = "CartPole-v1"
    """the id of the environment"""
    total_timesteps: int = 500000
    """total timesteps of the experiments"""
    learning_rate: float = 2.5e-4
    """the learning rate of the optimizer"""
    num_envs: int = 1
    """the number of parallel game environments"""
    buffer_size: int = 10000
    """the replay memory buffer size"""
    gamma: float = 0.99
    """the discount factor gamma"""
    tau: float = 1.0
    """the target network update rate"""
    target_network_frequency: int = 500
    """the timesteps it takes to update the target network"""
    batch_size: int = 128
    """the batch size of sample from the reply memory"""
    start_e: float = 1
    """the starting epsilon for exploration"""
    end_e: float = 0.05
    """the ending epsilon for exploration"""
    exploration_fraction: float = 0.5
    """the fraction of `total-timesteps` it takes from start-e to go end-e"""
    learning_starts: int = 10000
    """timestep to start learning"""
    train_frequency: int = 10
    """the frequency of training"""


def make_env(env_id, seed, idx, capture_video, run_name):
    def thunk():
        if capture_video and idx == 0:
            env = gym.make(env_id, render_mode="rgb_array")
            env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        else:
            env = gym.make(env_id)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        env.action_space.seed(seed)

        return env

    return thunk


# ALGO LOGIC: initialize agent here:
class QNetwork(nn.Module):
    def __init__(self, env):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(np.array(env.single_observation_space.shape).prod(), 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, env.single_action_space.n),
        )

    def forward(self, x):
        return self.network(x)


def linear_schedule(start_e: float, end_e: float, duration: int, t: int):
    slope = (end_e - start_e) / duration
    return max(slope * t + start_e, end_e)


if __name__ == "__main__":
    # args = tyro.cli(Args)
    args = tyro.cli(Args, args=["--env-id", "CartPole-v1", "--capture_video"])
    assert args.num_envs == 1, "vectorized envs are not supported at the moment"
    run_name = f"{args.env_id}__{args.exp_name}__{args.seed}__{int(time.time())}"
    if args.track:
        import wandb

        wandb.init(
            project=args.wandb_project_name,
            entity=args.wandb_entity,
            sync_tensorboard=True,
            config=vars(args),
            name=run_name,
            monitor_gym=True,
            save_code=True,
        )
    writer = SummaryWriter(f"runs/{run_name}")
    writer.add_text(
        "hyperparameters",
        "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(args).items()])),
    )

    # TRY NOT TO MODIFY: seeding
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.backends.cudnn.deterministic = args.torch_deterministic

    device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")

    # env setup
    envs = gym.vector.SyncVectorEnv(
        [make_env(args.env_id, args.seed + i, i, args.capture_video, run_name) for i in range(args.num_envs)]
    )
    assert isinstance(envs.single_action_space, gym.spaces.Discrete), "only discrete action space is supported"

    q_network = QNetwork(envs).to(device)
    optimizer = optim.Adam(q_network.parameters(), lr=args.learning_rate)
    target_network = QNetwork(envs).to(device)
    target_network.load_state_dict(q_network.state_dict())

    rb = ReplayBuffer(
        args.buffer_size,
        envs.single_observation_space,
        envs.single_action_space,
        device,
        handle_timeout_termination=False,
    )
    start_time = time.time()

    # TRY NOT TO MODIFY: start the game
    obs, _ = envs.reset(seed=args.seed)
    for global_step in range(args.total_timesteps):
        # ALGO LOGIC: put action logic here
        epsilon = linear_schedule(args.start_e, args.end_e, args.exploration_fraction * args.total_timesteps, global_step)
        if random.random() < epsilon:
            actions = np.array([envs.single_action_space.sample() for _ in range(envs.num_envs)])
        else:
            q_values = q_network(torch.Tensor(obs).to(device))
            actions = torch.argmax(q_values, dim=1).cpu().numpy()

        # TRY NOT TO MODIFY: execute the game and log data.
        next_obs, rewards, terminations, truncations, infos = envs.step(actions)

        # TRY NOT TO MODIFY: record rewards for plotting purposes
        if "final_info" in infos:
            for info in infos["final_info"]:
                if info and "episode" in info:
                    print(f"global_step={global_step}, episodic_return={info['episode']['r']}")
                    writer.add_scalar("charts/episodic_return", info["episode"]["r"], global_step)
                    writer.add_scalar("charts/episodic_length", info["episode"]["l"], global_step)

        # TRY NOT TO MODIFY: save data to reply buffer; handle `final_observation`
        real_next_obs = next_obs.copy()
        for idx, trunc in enumerate(truncations):
            if trunc:
                real_next_obs[idx] = infos["final_observation"][idx]
        rb.add(obs, real_next_obs, actions, rewards, terminations, infos)

        # TRY NOT TO MODIFY: CRUCIAL step easy to overlook
        obs = next_obs

        # ALGO LOGIC: training.
        if global_step > args.learning_starts:
            if global_step % args.train_frequency == 0:
                data = rb.sample(args.batch_size)
                with torch.no_grad():
                    target_max, _ = target_network(data.next_observations).max(dim=1)
                    td_target = data.rewards.flatten() + args.gamma * target_max * (1 - data.dones.flatten())
                old_val = q_network(data.observations).gather(1, data.actions).squeeze()
                loss = F.mse_loss(td_target, old_val)

                if global_step % 100 == 0:
                    writer.add_scalar("losses/td_loss", loss, global_step)
                    writer.add_scalar("losses/q_values", old_val.mean().item(), global_step)
                    print("SPS:", int(global_step / (time.time() - start_time)))
                    writer.add_scalar("charts/SPS", int(global_step / (time.time() - start_time)), global_step)

                # optimize the model
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            # update target network
            if global_step % args.target_network_frequency == 0:
                for target_network_param, q_network_param in zip(target_network.parameters(), q_network.parameters()):
                    target_network_param.data.copy_(
                        args.tau * q_network_param.data + (1.0 - args.tau) * target_network_param.data
                    )

    if args.save_model:
        model_path = f"runs/{run_name}/{args.exp_name}.cleanrl_model"
        torch.save(q_network.state_dict(), model_path)
        print(f"model saved to {model_path}")
        from cleanrl_utils.evals.dqn_eval import evaluate

        episodic_returns = evaluate(
            model_path,
            make_env,
            args.env_id,
            eval_episodes=10,
            run_name=f"{run_name}-eval",
            Model=QNetwork,
            device=device,
            epsilon=args.end_e,
        )
        for idx, episodic_return in enumerate(episodic_returns):
            writer.add_scalar("eval/episodic_return", episodic_return, idx)

        if args.upload_model:
            from cleanrl_utils.huggingface import push_to_hub

            repo_name = f"{args.env_id}-{args.exp_name}-seed{args.seed}"
            repo_id = f"{args.hf_entity}/{repo_name}" if args.hf_entity else repo_name
            push_to_hub(args, episodic_returns, repo_id, "DQN", f"runs/{run_name}", f"videos/{run_name}-eval")

    envs.close()
    writer.close()

/Users/shibo/Workspace/cleanrl/.venv/lib/python3.9/site-packages/tensorboard/compat/proto/histogram_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
/Users/shibo/Workspace/cleanrl/.venv/lib/python3.9/site-packages/tensorboard/compat/proto/histogram_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
/Users/shibo/Workspace/cleanrl/.venv/lib/python3.9/site-packages/tensorboard/compat/proto/histogram_pb2.py:29: DeprecationWarning: Call to deprecated create function Descriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated c

Moviepy - Building video /Users/shibo/Workspace/cleanrl/videos/CartPole-v1__dqn__1__1771880544/rl-video-episode-0.mp4.
Moviepy - Writing video /Users/shibo/Workspace/cleanrl/videos/CartPole-v1__dqn__1__1771880544/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /Users/shibo/Workspace/cleanrl/videos/CartPole-v1__dqn__1__1771880544/rl-video-episode-0.mp4
global_step=28, episodic_return=[29.]
Moviepy - Building video /Users/shibo/Workspace/cleanrl/videos/CartPole-v1__dqn__1__1771880544/rl-video-episode-1.mp4.
Moviepy - Writing video /Users/shibo/Workspace/cleanrl/videos/CartPole-v1__dqn__1__1771880544/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /Users/shibo/Workspace/cleanrl/videos/CartPole-v1__dqn__1__1771880544/rl-video-episode-1.mp4
global_step=38, episodic_return=[10.]
global_step=49, episodic_return=[11.]
global_step=85, episodic_return=[36.]
global_step=98, episodic_return=[13.]
global_step=114, episodic_return=[16.]
global_step=131, episodic_return=[17.]
global_step=150, episodic_return=[19.]


Moviepy - Building video /Users/shibo/Workspace/cleanrl/videos/CartPole-v1__dqn__1__1771880544/rl-video-episode-8.mp4.
Moviepy - Writing video /Users/shibo/Workspace/cleanrl/videos/CartPole-v1__dqn__1__1771880544/rl-video-episode-8.mp4



...

Moviepy - Done !
Moviepy - video ready /Users/shibo/Workspace/cleanrl/videos/CartPole-v1__dqn__1__1771880544/rl-video-episode-2000.mp4
global_step=54623, episodic_return=[13.]
global_step=54692, episodic_return=[69.]
SPS: 7405
global_step=54707, episodic_return=[15.]
global_step=54771, episodic_return=[64.]
SPS: 7408
global_step=54816, episodic_return=[45.]
global_step=54847, episodic_return=[31.]
global_step=54868, episodic_return=[21.]
SPS: 7410
global_step=54936, episodic_return=[68.]
global_step=54959, episodic_return=[23.]
SPS: 7412
global_step=55011, episodic_return=[52.]
global_step=55026, episodic_return=[15.]
global_step=55038, episodic_return=[12.]
SPS: 7415


global_step=55186, episodic_return=[148.]
global_step=55197, episodic_return=[11.]
SPS: 7417
global_step=55254, episodic_return=[57.]
global_step=55268, episodic_return=[14.]
SPS: 7419
global_step=55315, episodic_return=[47.]
global_step=55348, episodic_return=[33.]
global_step=55396, episodic_return=[48.]
SPS: 7421
global_step=55425, episodic_return=[29.]
global_step=55456, episodic_return=[31.]
global_step=55485, episodic_return=[29.]
SPS: 7423
global_step=55531, episodic_return=[46.]
global_step=55546, episodic_return=[15.]
global_step=55561, episodic_return=[15.]
SPS: 7425
global_step=55609, episodic_return=[48.]
global_step=55628, episodic_return=[19.]
global_step=55648, episodic_return=[20.]
global_step=55683, episodic_return=[35.]
SPS: 7426
global_step=55724, episodic_return=[41.]
global_step=55762, episodic_return=[38.]
SPS: 7429
global_step=55837, episodic_return=[75.]
global_step=55851, episodic_return=[14.]
global_step=55892, episodic_return=[41.]
SPS: 7431
global_step=55944

Moviepy - Done !
Moviepy - video ready /Users/shibo/Workspace/cleanrl/videos/CartPole-v1__dqn__1__1771880544/rl-video-episode-3000.mp4
global_step=106344, episodic_return=[105.]
global_step=106366, episodic_return=[22.]
SPS: 8008
global_step=106446, episodic_return=[80.]
global_step=106497, episodic_return=[51.]
SPS: 8009
global_step=106558, episodic_return=[61.]
SPS: 8009
SPS: 8011
SPS: 8011
global_step=106801, episodic_return=[243.]


global_step=106862, episodic_return=[61.]
SPS: 8012
global_step=106987, episodic_return=[125.]
SPS: 8012
global_step=107004, episodic_return=[17.]
SPS: 8013
global_step=107193, episodic_return=[189.]
SPS: 8013
SPS: 8014
global_step=107301, episodic_return=[108.]
SPS: 8015
global_step=107428, episodic_return=[127.]
global_step=107498, episodic_return=[70.]
SPS: 8016
global_step=107531, episodic_return=[33.]
SPS: 8016
global_step=107669, episodic_return=[138.]
global_step=107680, episodic_return=[11.]
SPS: 8018
global_step=107718, episodic_return=[38.]
global_step=107735, episodic_return=[17.]
global_step=107771, episodic_return=[36.]
SPS: 8018
global_step=107836, episodic_return=[65.]
global_step=107866, episodic_return=[30.]
SPS: 8019
global_step=107970, episodic_return=[104.]
global_step=107988, episodic_return=[18.]
SPS: 8021
global_step=108028, episodic_return=[40.]
SPS: 8022
global_step=108133, episodic_return=[105.]
SPS: 8023
global_step=108214, episodic_return=[81.]
SPS: 8024
SPS

Moviepy - Done !
Moviepy - video ready /Users/shibo/Workspace/cleanrl/videos/CartPole-v1__dqn__1__1771880544/rl-video-episode-4000.mp4
global_step=410622, episodic_return=[500.]
SPS: 8276
SPS: 8276
SPS: 8276
SPS: 8276
SPS: 8276
global_step=411122, episodic_return=[500.]
SPS: 8276
SPS: 8276
SPS: 8276
SPS: 8276
SPS: 8277
global_step=411622, episodic_return=[500.]
SPS: 8277
SPS: 8277
SPS: 8277
SPS: 8277
SPS: 8277
global_step=412122, episodic_return=[500.]
SPS: 8277
SPS: 8277
SPS: 8277
SPS: 8277
SPS: 8277
global_step=412622, episodic_return=[500.]
SPS: 8278
SPS: 8278
SPS: 8278
SPS: 8278
SPS: 8278
global_step=413122, episodic_return=[500.]
SPS: 8278
SPS: 8278
SPS: 8278
SPS: 8278
SPS: 8279
global_step=413622, episodic_return=[500.]
SPS: 8279
SPS: 8279
SPS: 8279
SPS: 8279
SPS: 8279
global_step=414122, episodic_return=[500.]
SPS: 8279
SPS: 8279
SPS: 8280
SPS: 8280
SPS: 8280
global_step=414622, episodic_return=[500.]
SPS: 8280
SPS: 8280
SPS: 8280
SPS: 8280
SPS: 8280
global_step=415122, episodic